### Importing Necessary Libraries required to read files and transform raw data

In [1]:
import pandas as pd
import glob
import os

### Reading all the info files and merging them into one file

In [2]:
in_files = glob.glob(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Assignment Files\*info.csv')
in_files = sorted(in_files)

info_files = []

for file in in_files:
    in_df = pd.read_csv(file, sep = ',', names = ['info', 'field', 'player_team', 'player_name', 'serial_no'])
    d1 = in_df
    d2 = d1.query('field == "season"')
    d3 = d2.set_index('field').T
    d4 = d3.iloc[1: , :]
    d5 = d4['season'].iloc[0]
    in_df.insert(0, 'tournament_season', d5)
    in_df = in_df.query('field == "player"')
    in_df = in_df.drop(['info', 'field', 'serial_no'], axis = 1)
    info_files.append(in_df)

### Reading the ball-by-ball files separately for batters and bowlers to get different datasets

In [3]:
bt_files = glob.glob(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Assignment Files\*[0-9].csv')
bt_files = sorted(bt_files)

batter_files = []

for file in bt_files:
    bt_df = pd.read_csv(file)
    batter_files.append(bt_df)

In [4]:
bw_files = glob.glob(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Assignment Files\*[0-9].csv')
bw_files = sorted(bw_files)

bowler_files = []

for file in bw_files:
    bw_df = pd.read_csv(file)
    bowler_files.append(bw_df)

### Performing minor data cleaning and merging the info files with the batting and bowling files. Saving the new files.

In [5]:
info_bat_files = []

for i in range(0, len(info_files)):
    ib_df = pd.merge(info_files[i], batter_files[i], left_on = 'player_name', right_on = 'striker', how = 'left')
    info_bat_files.append(ib_df)
    
batting_data = pd.concat(info_bat_files, ignore_index = True)

In [6]:
batting_data = batting_data.replace(to_replace=["Rising Pune Supergiant"],
           value="Rising Pune Supergiants")
batting_data = batting_data.replace(to_replace=["Kings XI Punjab"],
           value="Punjab Kings")
batting_data = batting_data.replace(to_replace=["Delhi Daredevils"],
           value="Delhi Capitals")

In [7]:
info_bowl_files = []

for i in range(0, len(info_files)):
    ib_df = pd.merge(info_files[i], bowler_files[i], left_on = 'player_name', right_on = 'bowler', how = 'left')
    info_bowl_files.append(ib_df)
    
bowling_data = pd.concat(info_bowl_files, ignore_index = True)

In [8]:
bowling_data = bowling_data.replace(to_replace=["Rising Pune Supergiant"],
           value="Rising Pune Supergiants")
batting_data = batting_data.replace(to_replace=["Kings XI Punjab"],
           value="Punjab Kings")
batting_data = batting_data.replace(to_replace=["Delhi Daredevils"],
           value="Delhi Capitals")

In [10]:
batting_data.to_csv(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Final Files\batting_data.csv', index = False)
bowling_data.to_csv(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Final Files\bowling_data.csv', index = False)

### Calculating the required metrics for batters' and bowlers'

In [11]:
bt_df = pd.read_csv(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Final Files\batting_data.csv')

C:\Users\DELL\AppData\Local\Temp\ipykernel_11896\3236746076.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  bt_df = pd.read_csv(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Final Files\batting_data.csv')


In [12]:
bat_table = bt_df.groupby(['player_name', 'player_team', 'tournament_season', 'match_id'], dropna = False).apply(lambda x: pd.Series(dict(
    no_of_matches = ((x.match_id).nunique() + (x.match_id).isnull().sum()),
    no_of_innings = (x.match_id).nunique(),
    total_runs = (x.runs_off_bat).sum(),
    balls_faced = (x.ball).count(),
    fours = (x.runs_off_bat == 4).sum(),
    sixes = (x.runs_off_bat == 6).sum(),
    outs = (x.striker == x.player_dismissed).count(),
    thirties = ((x.runs_off_bat).sum() >= 30).sum(),
    fifties = ((x.runs_off_bat).sum() >= 50).sum(),
    hundreds = ((x.runs_off_bat).sum() >= 100).sum(),
    not_outs = ((x.match_id).nunique() - (x.wicket_type).count())
    )))

In [13]:
bat_data = bat_table.groupby(['player_name', 'player_team', 'tournament_season']).agg(no_of_matches = ('no_of_matches', 'sum'),
                            no_of_innings = ('no_of_innings', 'sum'),
                            total_runs = ('total_runs', 'sum'),
                            balls_faced = ('balls_faced', 'sum'),
                            fours = ('fours', 'sum'),
                            sixes = ('sixes', 'sum'),
                            outs = ('outs', 'sum'),
                            thirties = ('thirties', 'sum'),
                            fifties = ('fifties', 'sum'),
                            hundreds = ('hundreds', 'sum'),
                            max_runs = ('total_runs', 'max'),
                            not_outs = ('not_outs', 'sum')
                            )

In [14]:
bw_df = pd.read_csv(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Final Files\bowling_data.csv')
bw_df[['over_no', 'ball_no']] = bw_df.ball.astype(str).str.split('.', expand = True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_11896\3668179717.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  bw_df = pd.read_csv(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Final Files\bowling_data.csv')


In [15]:
bowl_table = bw_df.groupby(['player_name', 'player_team', 'tournament_season', 'match_id', 'over_no'], dropna = False).apply(lambda x: pd.Series(dict(
    no_of_matches = ((x.match_id).nunique() + (x.match_id).isnull().sum()),
    no_of_innings = (x.match_id).nunique(),
    balls = (x.ball).count(),
    overs = (x.ball).count()%6/10 + (x.ball).count()//6,
    total_runs = (x.runs_off_bat.sum() + x.extras.sum()),
    wickets = (x.wicket_type).count(),
    three_wickets = ((x.wicket_type).count() >= 3).sum(),
    five_wickets = ((x.wicket_type).count() >= 5).sum(),
    dot_balls = ((x.runs_off_bat + x.extras) == 0).sum(),
    maidens = ((((x.runs_off_bat + x.extras) == 0).sum()) == 6).sum()
    )))

In [16]:
bowl_data = bowl_table.groupby(['player_name', 'player_team', 'tournament_season']).agg(no_of_matches = ('no_of_matches', 'sum'),
                            no_of_innings = ('no_of_innings', 'sum'),
                            balls = ('balls', 'sum'),
                            overs = ('overs', 'sum'),
                            total_runs = ('total_runs', 'sum'),
                            wickets = ('wickets', 'sum'),
                            three_wickets = ('three_wickets', 'sum'),
                            five_wickets = ('five_wickets', 'sum'),
                            dot_balls = ('dot_balls', 'sum'),
                            maidens = ('maidens', 'sum')
                            )

### Saving the final files to use as source data for the dashboard

In [17]:
bat_data.to_csv(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Final Files\batters_data.csv')
bowl_data.to_csv(r'C:\Users\DELL\Documents\Assignments\CricHeroes\Final Files\bowlers_data.csv')